In [1]:
import math
import datetime
import numpy as np
import random
import re

from olsq.input import input_qasm
from olsq.output import output_qasm
from olsq.device import qcdevice
import pkgutil

from olsq.solve import OLSQ
from olsq.solve import qcdevice

import qiskit
import qiskit.qasm2
from qiskit import QuantumCircuit,transpile
from qiskit_aer import AerSimulator


from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

In [2]:
def print_histo(counts):    
# Convert the dictionary keys to integers for sorting
    # Get all possible binary values
    all_binary_values = ['{:04b}'.format(i) for i in range(2**len(list(counts.keys())[0]))]

    # Create a dictionary with zero counts for all possible binary values
    full_counts = {binary_value: counts.get(binary_value, 0) for binary_value in all_binary_values}

    # Sort keys and values based on keys (binary representation)
    sorted_counts = {k: full_counts[k] for k in sorted(full_counts.keys())}

    # Plot the histogram
    plt.bar(sorted_counts.keys(), sorted_counts.values())

    # Set labels and title
    plt.xlabel('Measured binary strings')
    plt.ylabel('Counts')
    plt.title('Measurments')

    # Diagonally rotate x-axis labels
    plt.xticks(rotation=45, ha='right')

    # Show the plot
    plt.show()

def swap_to_cnot(qasm):


    # Regular expression to find the 'swaps q[x], q[y];' pattern
    pattern = r'swap q\[(\d+)\], q\[(\d+)\];'
    
    # Replacement function
    def replacement(match):
        x = match.group(1)
        y = match.group(2)
        return f'cx q[{x}], q[{y}]; cx q[{y}], q[{x}]; cx q[{x}], q[{y}];'
    
    # Perform the substitution
    result = re.sub(pattern, replacement, qasm)
    return result

def compare_distribuitions(counts1,counts2):

    counts_diff={}
    mean_diff=0
    tot_count=0
    KL=0

    for string in counts1.keys():
        counts_diff[string]=None

    for string in counts1.keys():
        tot_count+=counts1[string]

    for string in counts1.keys():
        counts_diff[string]=(abs(counts1[string]/tot_count-counts2[string]/tot_count))

    for string in counts1.keys():
        mean_diff+=counts_diff[string]

    for string in counts1.keys():
        KL+=(counts1[string]/tot_count)*math.log((counts1[string]/tot_count)/(counts2[string]/tot_count))


    return mean_diff/2, max(counts_diff.values()),KL



In [ ]:
def remove_redundant_swaps(t_new,pi_n,time_n,space_n,sigma_n):
    flag=1
    while(flag==1):
        flag=0
        for t in range(len(sigma_n[k][:])):
            for k in range(len(sigma_n[:][0])):
                if sigma_n[k][t]==sigma_n[k][t+1]==1:

                    for row in sigma_n:
                        del row[t]
                        del row[t+1]
                    
                    for row in pi_n:
                        del row[t]
                        del row[t+1]

                    del space_n[t]
                    del space_n[t+1]

                    del time_n[t]
                    del time_n[t+1]

                    t_new=-1
                    flag=1

                    

def genetic_algo(t_new_list,pi_n_list,time_n_list,space_n_list,sigma_n_list, t_new_sol,pi_n_sol,time_n_sol,space_n_sol,sigma_n_sol):

    #calculate the length of each solution
    #take the n bests
    
    #find pairs that share a swap location
    #if no pairs share a location terminate the algorithm with output the best solution between the ones in that generation

    #for each pair:
        #divide the pairs into 2 parts (before the sared location, after the shared location)
        #calculate the length of the 2 parts
        #take the best first and the best second 
        # merge them

    

    t_new_sol,pi_n_sol,time_n_sol,space_n_sol,sigma_n_sol = t_new_list[-1],pi_n_list[-1],time_n_list[-1],space_n_list[-1],sigma_n_list[-1]

    return  t_new_sol,pi_n_sol,time_n_sol,space_n_sol,sigma_n_sol

In [3]:
def collision_extracting(list_gate_qubits):
    """Extract collision relations between the gates,
    If two gates g_1 and g_2 both acts on a qubit (at different time),
    we say that g_1 and g_2 collide on that qubit, which means that
    (1,2) will be in collision list.

    Args:
        list_gate_qubits: a list of gates in OLSQ IR
    
    Returns:
        list_collision: a list of collisions between the gates
    """

    list_collision = list()
    # We sweep through all the gates.  For each gate, we sweep through all the
    # gates after it, if they both act on some qubit, append them in the list.
    for g in range(len(list_gate_qubits)):
        for gg in range(g + 1, len(list_gate_qubits)):
            
            if list_gate_qubits[g][0] == list_gate_qubits[gg][0]:
                    list_collision.append((g, gg))
                
            if len(list_gate_qubits[gg]) == 2:
                if list_gate_qubits[g][0] == list_gate_qubits[gg][1]:
                    list_collision.append((g, gg))
            
            if len(list_gate_qubits[g]) == 2:
                if list_gate_qubits[g][1] == list_gate_qubits[gg][0]:
                    list_collision.append((g, gg))
                if len(list_gate_qubits[gg]) == 2:
                    if list_gate_qubits[g][1] == list_gate_qubits[gg][1]:
                        list_collision.append((g, gg))
    
    return tuple(list_collision)

def dependency_extracting(list_gate_qubits, count_program_qubit: int):
    """Extract dependency relations between the gates.
    If two gates g_1 and g_2 both acts on a qubit *and there is no gate
    between g_1 and g_2 that act on this qubit*, we then say that
    g2 depends on g1, which means that (1,2) will be in dependency list.

    Args:
        list_gate_qubits: a list of gates in OLSQ IR
        count_program_qubit: the number of logical/program qubit
    
    Returns:
        list_dependency: a list of dependency between the gates
    """

    list_dependency = []
    list_last_gate = [-1 for i in range(count_program_qubit)]
    # list_last_gate records the latest gate that acts on each qubit.
    # When we sweep through all the gates, this list is updated and the
    # dependencies induced by the update is noted.
    for i, qubits in enumerate(list_gate_qubits):
        
        if list_last_gate[qubits[0]] >= 0:
            list_dependency.append((list_last_gate[qubits[0]], i))
        list_last_gate[qubits[0]] = i

        if len(qubits) == 2:
            if list_last_gate[qubits[1]] >= 0:
                list_dependency.append((list_last_gate[qubits[1]], i))
            list_last_gate[qubits[1]] = i

    return tuple(list_dependency)


class Heuristic(OLSQ):
    def __init__(self, objective_name, mode):
        """Set the objective of OLSQ, and whether it is transition-based

        Args:
            objective_name: can be "depth", "swap", or "fidelity"
            mode: can be "normal" or "transition" (TB-OLSQ in the paper)       
        """
        
        if objective_name == "depth":
            self.objective_name = objective_name
        elif objective_name == "swap":
            self.objective_name = objective_name
        elif objective_name == "fidelity":
            self.objective_name = objective_name
        else:
            raise ValueError("Invalid Objective Name")

        if mode == "transition":
            self.if_transition_based = True
        elif mode == "normal":
            self.if_transition_based = False
        else:
            raise ValueError("Invalid Choice of Transition-Based or Not")

        # These values should be updated in setdevice(...)
        self.device = None
        self.count_physical_qubit = 0
        self.list_qubit_edge = []
        self.swap_duration = 0

        # These values should be updated in setprogram(...)
        self.list_gate_qubits = []
        self.count_program_qubit = 0
        self.list_gate_name = []
        
        # bound_depth is a hyperparameter
        self.bound_depth = 0

        self.inpput_dependency = False
        self.list_gate_dependency = []

    def setdevice(self, device: qcdevice):
        """Pass in parameters from the given device.  If in TB mode,
           swap_duration is set to 1 without modifying the device.

        Args:
            device: a qcdevice object for OLSQ
        """
     
        self.device = device
        self.count_physical_qubit = device.count_physical_qubit
        self.list_qubit_edge = device.list_qubit_edge
        self.swap_duration = device.swap_duration
        if self.if_transition_based:
            self.swap_duration = 1

    def setprogram(self, program, input_mode: str = None):
        """Translate input program to OLSQ IR, and set initial depth
        An example of the intermediate representation is shown below.
        It contains three things: 1) the number of qubit in the program,
        2) a list of tuples representing qubit(s) acted on by a gate,
        the tuple has one index if it is a single-qubit gate,
        two indices if it is a two-qubit gate, and 3) a list of
        type/name of each gate, which is not important to OLSQ,
        and only needed when generating output.
        If in TB mode, initial depth=1; in normal mode, we perform ASAP
        scheduling without consideration of SWAP to calculate depth.

        Args:
            program: a qasm string, or a list of the three things in IR.
            input_mode: (optional) can be "IR" if the input has ben
                translated to OLSQ IR; can be "benchmark" to use one of
                the benchmarks.  Default mode assumes qasm input.

        Example:
            For the following circuit
                q_0: ───────────────────■───
                                        │  
                q_1: ───────■───────────┼───
                     ┌───┐┌─┴─┐┌─────┐┌─┴─┐
                q_2: ┤ H ├┤ X ├┤ TDG ├┤ X ├─
                     └───┘└───┘└─────┘└───┘ 
            count_program_qubit = 3
            gates = ((2,), (1,2), (2,), (0,1))
            gate_spec = ("h", "cx", "tdg", "cx")
        """
        
        if input_mode == "IR":
            self.count_program_qubit = program[0]
            self.list_gate_qubits = program[1]
            self.list_gate_name = program[2]
        elif input_mode == "benchmark":
            f = pkgutil.get_data(__name__, "benchmarks/" + program + ".qasm")
            program = input_qasm(f.decode("utf-8"))
            self.count_program_qubit = program[0]
            self.list_gate_qubits = program[1]
            self.list_gate_name = program[2]
        else:
            program = input_qasm(program)
            self.count_program_qubit = program[0]
            self.list_gate_qubits = program[1]
            self.list_gate_name = program[2]

        # calculate the initial depth
        if self.if_transition_based:
            self.bound_depth = 1
        else:
            push_forward_depth = [0 for i in range(self.count_program_qubit)]
            for qubits in self.list_gate_qubits:
                if len(qubits) == 1:
                    push_forward_depth[qubits[0]] += 1
                else:
                    tmp_depth = push_forward_depth[qubits[0]]
                    if tmp_depth < push_forward_depth[qubits[1]]:
                        tmp_depth = push_forward_depth[qubits[1]]
                    push_forward_depth[qubits[1]] = tmp_depth + 1
                    push_forward_depth[qubits[0]] = tmp_depth + 1
            self.bound_depth = max(push_forward_depth)

    def setdependency(self, dependency: list):
        """Specify dependency (non-commutation)

        Args:
            dependency: a list of gate index pairs
        
        Example:
            For the following circuit
                q_0: ───────────────────■───
                                        │  
                q_1: ───────■───────────┼───
                     ┌───┐┌─┴─┐┌─────┐┌─┴─┐
                q_2: ┤ H ├┤ X ├┤ TDG ├┤ X ├─
                     └───┘└───┘└─────┘└───┘ 
                gate   0    1     2     3
            dependency = [(0,1), (1,2), (2,3)]

            However, for this QAOA subcircuit (ZZ gates may have phase
            parameters, but we neglect them for simplicity here)
                         ┌──┐ ┌──┐
                q_0: ────┤ZZ├─┤  ├─
                     ┌──┐└┬─┘ │ZZ│  
                q_1: ┤  ├─┼───┤  ├─
                     │ZZ│┌┴─┐ └──┘
                q_2: ┤  ├┤ZZ├──────
                     └──┘└──┘ 
                gate   0   1   2
            dependency = []    # since ZZ gates are commutable
        """
        self.list_gate_dependency = dependency
        self.inpput_dependency = True

    def solve(self, output_mode: str = None, output_file_name: str = None):
        """Formulate an SMT, pass it to z3 solver, and output results.
        CORE OF OLSQ, EDIT WITH CARE.

        Args:
            output_mode: "IR" or left to default.
            output_file_name: a file to store the IR output or qasm.
        
        Returns:
            a list of results depending on output_mode
            "IR": 
            | list_scheduled_gate_name: name/type of each gate
            | list_scheduled_gate_qubits: qubit(s) each gate acts on
            | final_mapping: logical qubit |-> physical qubit in the end 
            | objective_value: depth/#swap/fidelity depending on setting
            None:
              a qasm string
              final_mapping
              objective_value
        """

        objective_name = self.objective_name
        device = self.device
        list_gate_qubits = self.list_gate_qubits
        count_program_qubit = self.count_program_qubit
        list_gate_name = self.list_gate_name
        count_physical_qubit = self.count_physical_qubit
        list_qubit_edge = self.list_qubit_edge
        swap_duration = self.swap_duration
        bound_depth = self.bound_depth

        #print("list_gate_qubits")
        #print(list_gate_qubits)

        #print("list_gate_name")
        #substitute cnot cx

        list_gate_name=[s.replace("cnot","cx") for s in list_gate_name]

        #print(list_gate_name)
        
        # pre-processing

        count_qubit_edge = len(list_qubit_edge)
        count_gate = len(list_gate_qubits)
        if self.objective_name == "fidelity":
            list_logfidelity_single = [
                int(1000 * math.log(device.list_fidelity_single[n]))
                for n in range(count_physical_qubit)]
            list_logfidelity_two = [
                int(1000 * math.log(device.list_fidelity_two[k]))
                for k in range(count_qubit_edge)]
            list_logfidelity_measure = [
                int(1000 * math.log(device.list_fidelity_measure[n]))
                for n in range(count_physical_qubit)]
        list_gate_two = list()
        list_gate_single = list()
        for l in range(count_gate):
            if len(list_gate_qubits[l]) == 1:
                list_gate_single.append(l)
            else:
                list_gate_two.append(l)

        # list_adjacency_qubit takes in a physical qubit index _p_, and
        # returns the list of indices of physical qubits adjacent to _p_
        list_adjacent_qubit = list()
        # list_span_edge takes in a physical qubit index _p_,
        # and returns the list of edges spanned from _p_
        list_span_edge = list()
        for n in range(count_physical_qubit):
            list_adjacent_qubit.append(list())
            list_span_edge.append(list())
        for k in range(count_qubit_edge):
            list_adjacent_qubit[list_qubit_edge[k][0]].append(list_qubit_edge[k][1])
            list_adjacent_qubit[list_qubit_edge[k][1]].append(list_qubit_edge[k][0])
            list_span_edge[list_qubit_edge[k][0]].append(k)
            list_span_edge[list_qubit_edge[k][1]].append(k)

        # if_overlap_edge takes in two edge indices _e_ and _e'_,
        # and returns whether or not they overlap
        if_overlap_edge = [[0] * count_qubit_edge
            for k in range(count_qubit_edge)]
        # list_over_lap_edge takes in an edge index _e_,
        # and returnsthe list of edges that overlap with _e_
        list_overlap_edge = list()
        # list_count_overlap_edge is the list of lengths of
        # overlap edge lists of all the _e_
        list_count_overlap_edge = list()
        for k in range(count_qubit_edge):
            list_overlap_edge.append(list())
        for k in range(count_qubit_edge):
            for kk in range(k + 1, count_qubit_edge):
                if (   (list_qubit_edge[k][0] == list_qubit_edge[kk][0]
                        or list_qubit_edge[k][0] == list_qubit_edge[kk][1])
                    or (list_qubit_edge[k][1] == list_qubit_edge[kk][0]
                        or list_qubit_edge[k][1] == list_qubit_edge[kk][1]) ):
                    list_overlap_edge[k].append(kk)
                    list_overlap_edge[kk].append(k)
                    if_overlap_edge[kk][k] = 1
                    if_overlap_edge[k][kk] = 1
        for k in range(count_qubit_edge):
            list_count_overlap_edge.append(len(list_overlap_edge[k]))

        if not self.inpput_dependency:
            list_gate_dependency = collision_extracting(list_gate_qubits)
        else:
            list_gate_dependency = self.list_gate_dependency

        # index function: takes two physical qubit indices _p_ and _p'_,
        # and returns the index of the edge between them if there is one
        map_edge_index = [[0] * count_physical_qubit] * count_physical_qubit
        for k in range(count_qubit_edge):
            map_edge_index[list_qubit_edge[k][0]][list_qubit_edge[k][1]] = k
            map_edge_index[list_qubit_edge[k][1]][list_qubit_edge[k][0]] = k

        not_solved = True
        start_time = datetime.datetime.now()
    
        ############################################################################################################

        # variable setting 

        # at cycle t, logical qubit q is mapped to pi[q][t]
        #pi =np.zeros((count_program_qubit,bound_depth), dtype=int)
        pi = [[] for _ in range(count_program_qubit)]

        # time coordinate for gate l is time[l]
        time = np.zeros(count_gate, dtype=int)
        #time=[]

        # Create an array of tuples
        #space = [(0, 0) for i in range(count_gate)]
        space =[]
        

        # if at cycle t, a SWAP finishing on edge k, then sigma[k][t]=1
        #sigma = np.zeros((count_qubit_edge, bound_depth), dtype=int)
        sigma = [[] for _ in range(count_qubit_edge)]

        # for depth optimization
        #depth = Int('depth')

        # for swap optimization
        #count_swap = Int('num_swap')
        
        """
            # for fidelity optimization
        if objective_name == "fidelity":
            u = [Int("num_1qbg_p{}".format(n))
                 for n in range(count_physical_qubit)]
            v = [Int("num_2qbg_e{}".format(k))
                 for k in range(count_qubit_edge)]
            vv = [Int("num_swap_e{}".format(k))
                  for k in range(count_qubit_edge)]
            w = [Int("num_meas_p{}".format(n))
                 for n in range(count_physical_qubit)]
            fidelity = Int('log_fidelity')

        """
        #lsqc = Optimize()


        
        #for k in range (len(list_gate_qubits)):
            #space[k] = list_gate_qubits[k]
        #    space.append(list_gate_qubits[k])
        ##########################################################################################################

        for l in range(count_gate):
            if l in list_gate_single:
                 #space[l] = list_gate_qubits[l][0]
                 space.append((list_gate_qubits[l][0]))    
            elif l in list_gate_two:
                #space[l] = (list_gate_qubits[l][0],list_gate_qubits[l][1])
                space.append((list_gate_qubits[l][0],list_gate_qubits[l][1]))

        #print("space")
        #print(space)

        dep_extract=dependency_extracting(list_gate_qubits, count_program_qubit)

        for k in range (len(list_gate_qubits)):
            for kk in range (0,k):
                if (kk,k) in dep_extract:
                    
                    time[k] = time[kk]+1
                    #time.append(time[kk]+1)
                #else if k has no dependency with any other gate the time[l] is not modify and remain 0 
        #print("time")
        #print(time)


        for t in range(bound_depth):
            for q in range(count_program_qubit):
                #pi[q][t]=q
                pi[q].append(q)


        for t in range(bound_depth):
            for e in range(count_qubit_edge):
                #pi[q][t]=q
                sigma[e].append(0)
        
        ###################################################################################################

        initial_population=100

                # at cycle t, logical qubit q is mapped to pi[q][t]
        #pi =np.zeros((count_program_qubit,bound_depth), dtype=int)
        pi_n = [[] for _ in range(count_program_qubit)]
        pi_n_sol = [[] for _ in range(count_program_qubit)]
        pi_n_list = [[[] for _ in range(count_program_qubit)] for _ in range(initial_population)]
        # time coordinate for gate l is time[l]
        time_n = []
        time_n_sol = []
        time_n_list=[[] for _ in range(initial_population)]
        #time=[]

        # Create an array of tuples
        #space = [(0, 0) for i in range(count_gate)]
        space_n =[]
        space_n_sol =[]
        space_n_list=[[] for _ in range(initial_population)]
        

        # if at cycle t, a SWAP finishing on edge k, then sigma[k][t]=1
        #sigma = np.zeros((count_qubit_edge, bound_depth), dtype=int)
        sigma_n = [[] for _ in range(count_qubit_edge)]
        sigma_n_sol = [[] for _ in range(count_qubit_edge)]
        sigma_n_list = [[[] for _ in range(count_program_qubit)] for _ in range(initial_population)]

        t_new_list=np.zeros(initial_population)

        list_scheduled_gate_name_new=[]

        #initial mapping is the same as logical mapping
        for q in range(count_program_qubit):
            pi_n[q].append(pi[q][0])

        #scheduling
        t_new=-1
        for t in range(max(time)+1):
            t_new+=1
            for g in range(count_gate):
                if time[g]==t:
                    if g in list_gate_single:
                        q_logical=space[g]
                        q_physical=pi_n[q_logical][t_new-1]
                        space_n.append(q_physical)
                        time_n.append(t_new)
                        #list_scheduled_gate_name_new[t_new].append(list_scheduled_gate_name(g))
                    if g in list_gate_two:
                        s0_logical = list_gate_qubits[g][0]
                        r_logical= list_gate_qubits[g][1]

                        for i in range (initial_population):
                            pi_n_list[i][:].append(pi_n[:][-1])
                            space_n_list[i].append(space_n[-1])
                            sigma_n_list[i][:].append(sigma_n[:][-1])
                            time_n_list.append(time_n[-1])


                        for i in range (initial_population):

                            t_new_list[i]=t_new

                            while (not((pi_n_list[i][s0_logical][-1] , pi_n_list[i][r_logical][-1]) in list_qubit_edge) and 
                                not((pi_n_list[i][r_logical][-1], pi_n_list[i][s0_logical][-1]) in list_qubit_edge)):
                                
                                r_new_logical = 0
                                #is the new random proposed r_new_logical physical connected with r_logical
                                
                                #check if the proposed qubit is a valid choice to do the swap with
                                while (not((pi_n_list[i][r_new_logical][-1], pi_n_list[i][r_logical][-1])in list_qubit_edge) and
                                    not((pi_n_list[i][r_logical][-1], pi_n_list[i][r_new_logical][-1])) in list_qubit_edge):
                                    
                                    #propose a qubit to do the swap
                                    r_new_logical = random.randint(0, count_program_qubit-1)

                                #insert SWAP(new_r,r)
                                #assign pi, the pi of the swaps will be inverted all the other one remain the same
                                pi_n_list[i][r_logical].append(pi_n_list[i][r_new_logical][-1])
                                pi_n_list[i][r_new_logical].append(pi_n_list[i][r_logical][-1])
                                
                                #if qubit are not involved in the swap, the map doesn't change
                                for q in range(count_program_qubit):
                                    if (q!=r_logical and q!=r_new_logical):
                                        pi_n_list[i][q].append(pi_n_list[i][q][-1])

                                #assign values of sigma, if k is the edge where I inserted the swap append 1 otherwise 0
                                for k in range(len(list_qubit_edge)):
                                    if k==list_qubit_edge.index((min(pi_n_list[i][r_new_logical][t_new_list[i]],pi_n_list[i][r_logical][t_new_list[i]]),
                                                                 max(pi_n_list[i][r_new_logical][t_new_list[i]],pi_n_list[i][r_logical][t_new_list[i]]))):
                                        sigma_n_list[i][k].append(1)
                                    else:
                                        sigma_n_list[i][k].append(0)

                                #assign space, time of swap
                                space_n_list[i].append((pi_n_list[i][r_logical][t_new_list[i]],pi_n_list[i][r_new_logical][t_new_list[i]]))
                                time_n_list[i].append(t_new_list[i])
                                t_new_list[i]+=1

                                #r_logical=r_new_logical
                                #list_scheduled_gate_name_new[t_new].append("SWAP")
                                remove_redundant_swaps(t_new_list[i],pi_n_list[i],time_n_list[i],space_n_list[i],sigma_n_list[i])
                            
                            t_new_sol,pi_n_sol,time_n_sol,space_n_sol,sigma_n_sol= genetic_algo(t_new_list,pi_n_list,time_n_list,space_n_list,sigma_n_list)


                            #maybe here I need [:] after the lists
                            t_new.append(t_new_sol)
                            time_n.append(time_n_sol)
                            space_n.append(space_n_sol)

                            for q in range(len(count_program_qubit)):
                                pi_n[q].append(pi_n_sol[q])

                            for k in range(len(list_qubit_edge)):
                                sigma_n[k].append(sigma_n_sol[k])

                        
                        #insertion of the gate that now after the swap insertion is valid
                        q_logical=space[g]
                        q_physical_1=pi_n[q_logical[0]][t_new-1]
                        q_physical_2=pi_n[q_logical[1]][t_new-1]
                        time_n.append(t_new)
                        space_n.append((q_physical_1,q_physical_2))
                        #list_scheduled_gate_name_new[t_new].append(list_scheduled_gate_name(g))

            for q in range(count_program_qubit):
                if t_new!=0:
                    pi_n[q].append(pi_n[q][t_new-1])
                            
            for e in range(len(list_qubit_edge)):
                sigma_n[e].append(0)
                                
	
        """		        
        print("time_n")
        print(time_n)

        print("space_n")
        print(space_n)

        print("pi_n")
        print(pi_n)

        print("sigma_n")
        print(sigma_n)
        """

        compilation_time=datetime.datetime.now() - start_time

        print(f"Compilation time = {compilation_time}.")
        #model = lsqc.model()

        # post-processing

        result_time = []
        #result_depth = depth
        result_depth = int(max(time_n))
        count_gate_new=len(time_n)

        #if is not a swap append it on result_time
        #SWAPs will be added later separately
        #I sum up the column of sigma_n, if one of them is one a Swap occured otherwise it was not a Swap   
        sum_col = [sum(values) for values in zip(*sigma_n)]

        for l in range(len(time_n)):

#            if (sum_col[time_n[l]]==0 or (sum_col[time_n[l]]==1 and sum_col[time_n[l-1]]==1)):
#                result_time.append(time_n[l])

            #if is not a swap or it is a single qubit gate
            #there was problem that single gate after swap were in same interval as swap,
            #so assigned to the swap list
            if (sum_col[time_n[l]]==0 or isinstance(space_n[l], int)):
                result_time.append(time_n[l])

        list_result_swap = []
        for k in range(count_qubit_edge):
            for t in range(result_depth):
                if sigma_n[k][t]:
                    list_result_swap.append((k, t))
                    #print(f"SWAP on physical edge ({list_qubit_edge[k][0]},"+ f"{list_qubit_edge[k][1]}) at time {t}")

        """           
        for l in range(count_gate):
            if len(list_gate_qubits[l]) == 1:
                qq = list_gate_qubits[l][0]
                tt = result_time[l]
                #print(f"Gate {l}: {list_gate_name[l]} {qq} on qubit "+ f"{pi[qq][tt]} at time {tt}")
            else:
                qq = list_gate_qubits[l][0]
                qqq = list_gate_qubits[l][1]
                tt = result_time[l]
                #print(f"Gate {l}: {list_gate_name[l]} {qq}, {qqq} on qubits "+ f"{pi[qq][tt]} and "+ f"{pi[qqq][tt]} at time {tt}")
        """
        
        objective_value = 0
        if objective_name == "fidelity":
            objective_value = fidelity
            print(f"result fidelity = {math.exp(objective_value / 1000.0)}")
        elif objective_name == "swap":
            objective_value = len(list_result_swap)
            print(f"result additional SWAP count = {objective_value}.")
        else:
            objective_value = "depth"
            #print(f"result circuit depth = {objective_value}.")
        


        list_scheduled_gate_qubits = [[] for i in range(result_depth+1)]
        list_scheduled_gate_name = [[] for i in range(result_depth+1)]
        result_depth = 0

        
        #schedule SWAPs
        for (k, t) in list_result_swap:
            q0 = list_qubit_edge[k][0]
            q1 = list_qubit_edge[k][1]
            if self.swap_duration == 1:
                list_scheduled_gate_qubits[t].append((q0, q1))
                list_scheduled_gate_name[t].append("swap")
            elif self.swap_duration == 3:
                list_scheduled_gate_qubits[t].append((q0, q1))
                list_scheduled_gate_name[t].append("cx")
                list_scheduled_gate_qubits[t].append((q1, q0))
                list_scheduled_gate_name[t].append("cx")
                list_scheduled_gate_qubits[t].append((q0, q1))
                list_scheduled_gate_name[t].append("cx")
            else:
                raise ValueError("Expect SWAP duration one, or three")


        #schedule non SWAPs
        for l in range(count_gate):
            t = int(result_time[l])

            if result_depth < t + 1:
                result_depth = t + 1

            list_scheduled_gate_name[t].append(list_gate_name[l])
            if len(list_gate_qubits[l]) == 1:
                #problem this is the new list with swaps
                # so acces another element with possibility to have 2 arguments
                q_logical= space[l]
                q_physical=pi_n[q_logical][t]
                list_scheduled_gate_qubits[t].append((q_physical,))
            else:
                [q0, q1] = list_gate_qubits[l]
                tmp_t = t
                q0 = pi_n[q0][tmp_t]
                q1 = pi_n[q1][tmp_t]
                list_scheduled_gate_qubits[t].append((q0, q1))








        final_mapping = []
        for m in range(count_program_qubit):
            tmp_depth = result_depth - 1
            final_mapping.append(pi_n[m][tmp_depth])


        

        """
        print("list_scheduled_gate_name ")
        print(list_scheduled_gate_name)
        print(" ")
        print("list_scheduled_gate_qubits ")
        print(list_scheduled_gate_qubits)
        """

        qasm=output_qasm(device, result_depth, list_scheduled_gate_name,
                            list_scheduled_gate_qubits, final_mapping,
                            True, output_file_name)

        #print(qasm)
        circuit = QuantumCircuit()
        circuit = circuit.from_qasm_str(str(qasm))
        #print(circuit)
        #circuit.draw("mpl")



        
        """return (output_qasm(device, result_depth, list_scheduled_gate_name,
                                list_scheduled_gate_qubits, final_mapping,
                                True, output_file_name),
                    final_mapping,
                    objective_value)
        """
        
        return(result_depth,
               list_scheduled_gate_name,
               list_scheduled_gate_qubits,
               final_mapping,
               objective_value,
               qasm,
               compilation_time)

In [4]:
qasm_input_QAOA_3='''
OPENQASM 2.0;
include "qelib1.inc";

qreg q[3];
creg c[3];

// Initial state preparation: Apply Hadamard gates to all qubits
h q[0];
h q[1];
h q[2];

rz(0) q[0];
cx q[0], q[1];
rz(1) q[1];

cx q[0], q[2];
rz(2) q[2];

rz(1.3) q[0];
cx q[0], q[1];

cx q[0], q[2];
rz(1.3) q[1];
rz(1.3) q[2];

rz(1.3) q[0];


// Measurement
measure q[0] -> c[0];
measure q[1] -> c[1];
measure q[2] -> c[2];
'''

qasm_input_QAOA_4='''
OPENQASM 2.0;
include "qelib1.inc";

qreg q[4];
creg c[4];

// Initial state preparation: Apply Hadamard gates to all qubits
h q[0];
h q[1];
h q[2];
h q[3];

rz(0) q[0];
cx q[0], q[1];
rz(1) q[1];

cx q[0], q[2];
rz(2) q[2];

cx q[0], q[3];
rz(3) q[3];

rz(1.3) q[0];
cx q[0], q[1];

cx q[0], q[2];

rz(1.3) q[1];
rz(1.3) q[2];

cx q[0], q[3];
rz(1.3) q[3];
rz(1.3) q[0];

// Measurement
measure q[0] -> c[0];
measure q[1] -> c[1];
measure q[2] -> c[2];
measure q[3] -> c[3];
'''

qasm_input_QAOA_5='''
OPENQASM 2.0;
include "qelib1.inc";

qreg q[5];
creg c[5];

// Initial state preparation: Apply Hadamard gates to all qubits
h q[0];
h q[1];
h q[2];
h q[3];
h q[4];

rz(0) q[0];
cx q[0], q[1];
rz(1) q[1];

cx q[0], q[2];
rz(2) q[2];

cx q[0], q[3];
rz(3) q[3];

cx q[0], q[4];
rz(1.3) q[4];

rz(1.3) q[0];
cx q[0], q[1];

cx q[0], q[2];

rz(1.3) q[1];
rz(1.3) q[2];

cx q[0], q[3];
rz(1.3) q[3];

cx q[0], q[4];
rz(1.3) q[4];
rz(1.3) q[0];

// Measurement
measure q[0] -> c[0];
measure q[1] -> c[1];
measure q[2] -> c[2];
measure q[3] -> c[3];
measure q[4] -> c[4];
'''
qasm_input_QAOA_6='''

OPENQASM 2.0;
include "qelib1.inc";

qreg q[6];
creg c[6];

// Initial state preparation: Apply Hadamard gates to all qubits
h q[0];
h q[1];
h q[2];
h q[3];
h q[4];
h q[5];

rz(0) q[0];

cx q[0], q[1];
rz(1) q[1];

cx q[0], q[2];
rz(2) q[2];

cx q[0], q[3];
rz(3) q[3];

cx q[0], q[4];
rz(1.3) q[4];

cx q[0], q[5];
rz(1.3) q[5];

rz(1.3) q[0];

cx q[0], q[1];
rz(1.3) q[1];

cx q[0], q[2];
rz(1.3) q[2];

cx q[0], q[3];
rz(1.3) q[3];

cx q[0], q[4];
rz(1.3) q[4];

cx q[0], q[5];
rz(1.3) q[5];

rz(1.3) q[0];

// Measurement
measure q[0] -> c[0];
measure q[1] -> c[1];
measure q[2] -> c[2];
measure q[3] -> c[3];
measure q[4] -> c[4];
measure q[5] -> c[5];
'''

qasm_input_QAOA_7='''
OPENQASM 2.0;
include "qelib1.inc";

qreg q[7];
creg c[7];

// Initial state preparation: Apply Hadamard gates to all qubits
h q[0];
h q[1];
h q[2];
h q[3];
h q[4];
h q[5];
h q[6];

rz(0) q[0];

cx q[0], q[1];
rz(1) q[1];

cx q[0], q[2];
rz(2) q[2];

cx q[0], q[3];
rz(3) q[3];

cx q[0], q[4];
rz(1.3) q[4];

cx q[0], q[5];
rz(1.3) q[5];

cx q[0], q[6];
rz(1.3) q[6];


rz(1.3) q[0];

cx q[0], q[1];
rz(1.3) q[1];

cx q[0], q[2];
rz(1.3) q[2];

cx q[0], q[3];
rz(1.3) q[3];

cx q[0], q[4];
rz(1.3) q[4];

cx q[0], q[5];
rz(1.3) q[5];

cx q[0], q[6];
rz(1.3) q[6];


rz(1.3) q[0];

// Measurement
measure q[0] -> c[0];
measure q[1] -> c[1];
measure q[2] -> c[2];
measure q[3] -> c[3];
measure q[4] -> c[4];
measure q[5] -> c[5];
measure q[6] -> c[6];
'''



qasm_input_QAOA_8='''
OPENQASM 2.0;
include "qelib1.inc";

qreg q[8];
creg c[8];

// Initial state preparation: Apply Hadamard gates to all qubits
h q[0];
h q[1];
h q[2];
h q[3];
h q[4];
h q[5];
h q[6];
h q[7];

rz(0) q[0];

cx q[0], q[1];
rz(1) q[1];

cx q[0], q[2];
rz(2) q[2];

cx q[0], q[3];
rz(3) q[3];

cx q[0], q[4];
rz(1.3) q[4];

cx q[0], q[5];
rz(1.3) q[5];

cx q[0], q[6];
rz(1.3) q[6];

cx q[0], q[7];
rz(1.3) q[7];


rz(1.3) q[0];

cx q[0], q[1];
rz(1.3) q[1];

cx q[0], q[2];
rz(1.3) q[2];

cx q[0], q[3];
rz(1.3) q[3];

cx q[0], q[4];
rz(1.3) q[4];

cx q[0], q[5];
rz(1.3) q[5];

cx q[0], q[6];
rz(1.3) q[6];

cx q[0], q[7];
rz(1.3) q[7];

rz(1.3) q[0];

// Measurement
measure q[0] -> c[0];
measure q[1] -> c[1];
measure q[2] -> c[2];
measure q[3] -> c[3];
measure q[4] -> c[4];
measure q[5] -> c[5];
measure q[6] -> c[6];
measure q[7] -> c[7];
'''

qasm_input_QAOA_9='''
OPENQASM 2.0;
include "qelib1.inc";

qreg q[9];
creg c[9];

// Initial state preparation: Apply Hadamard gates to all qubits
h q[0];
h q[1];
h q[2];
h q[3];
h q[4];
h q[5];
h q[6];
h q[7];
h q[8];

rz(0) q[0];

cx q[0], q[1];
rz(1) q[1];

cx q[0], q[2];
rz(2) q[2];

cx q[0], q[3];
rz(3) q[3];

cx q[0], q[4];
rz(1.3) q[4];

cx q[0], q[5];
rz(1.3) q[5];

cx q[0], q[6];
rz(1.3) q[6];

cx q[0], q[7];
rz(1.3) q[7];

cx q[0], q[8];
rz(1.3) q[8];


rz(1.3) q[0];

cx q[0], q[1];
rz(1.3) q[1];

cx q[0], q[2];
rz(1.3) q[2];

cx q[0], q[3];
rz(1.3) q[3];

cx q[0], q[4];
rz(1.3) q[4];

cx q[0], q[5];
rz(1.3) q[5];

cx q[0], q[6];
rz(1.3) q[6];

cx q[0], q[7];
rz(1.3) q[7];

cx q[0], q[8];
rz(1.3) q[8];


rz(1.3) q[0];

// Measurement
measure q[0] -> c[0];
measure q[1] -> c[1];
measure q[2] -> c[2];
measure q[3] -> c[3];
measure q[4] -> c[4];
measure q[5] -> c[5];
measure q[6] -> c[6];
measure q[7] -> c[7];
measure q[8] -> c[8];
'''

qasm_input_QAOA_10='''
OPENQASM 2.0;
include "qelib1.inc";

qreg q[10];
creg c[10];

// Initial state preparation: Apply Hadamard gates to all qubits
h q[0];
h q[1];
h q[2];
h q[3];
h q[4];
h q[5];
h q[6];
h q[7];
h q[8];
h q[9];

rz(0) q[0];

cx q[0], q[1];
rz(1) q[1];

cx q[0], q[2];
rz(2) q[2];

cx q[0], q[3];
rz(3) q[3];

cx q[0], q[4];
rz(1.3) q[4];

cx q[0], q[5];
rz(1.3) q[5];

cx q[0], q[6];
rz(1.3) q[6];

cx q[0], q[7];
rz(1.3) q[7];

cx q[0], q[8];
rz(1.3) q[8];

cx q[0], q[9];
rz(1.3) q[9];


rz(1.3) q[0];

cx q[0], q[1];
rz(1.3) q[1];

cx q[0], q[2];
rz(1.3) q[2];

cx q[0], q[3];
rz(1.3) q[3];

cx q[0], q[4];
rz(1.3) q[4];

cx q[0], q[5];
rz(1.3) q[5];

cx q[0], q[6];
rz(1.3) q[6];

cx q[0], q[7];
rz(1.3) q[7];

cx q[0], q[8];
rz(1.3) q[8];

cx q[0], q[9];
rz(1.3) q[9];

rz(1.3) q[0];

// Measurement
measure q[0] -> c[0];
measure q[1] -> c[1];
measure q[2] -> c[2];
measure q[3] -> c[3];
measure q[4] -> c[4];
measure q[5] -> c[5];
measure q[6] -> c[6];
measure q[7] -> c[7];
measure q[8] -> c[8];
measure q[9] -> c[9];
'''

qasm_input_QAOA_15='''

OPENQASM 2.0;
include "qelib1.inc";

qreg q[15];
creg c[15];

// Initial state preparation: Apply Hadamard gates to all qubits
h q[0];
h q[1];
h q[2];
h q[3];
h q[4];
h q[5];
h q[6];
h q[7];
h q[8];
h q[9];
h q[10];
h q[11];
h q[12];
h q[13];
h q[14];

rz(0) q[0];

cx q[0], q[1];
rz(1) q[1];

cx q[0], q[2];
rz(2) q[2];

cx q[0], q[3];
rz(3) q[3];

cx q[0], q[4];
rz(1.3) q[4];

cx q[0], q[5];
rz(1.3) q[5];

cx q[0], q[6];
rz(1.3) q[6];

cx q[0], q[7];
rz(1.3) q[7];

cx q[0], q[8];
rz(1.3) q[8];

cx q[0], q[9];
rz(1.3) q[9];

cx q[0], q[10];
rz(1.3) q[10];

cx q[0], q[11];
rz(1.3) q[11];

cx q[0], q[12];
rz(1.3) q[12];

cx q[0], q[13];
rz(1.3) q[13];

cx q[0], q[14];
rz(1.3) q[14];


rz(1.3) q[0];

cx q[0], q[1];
rz(1.3) q[1];

cx q[0], q[2];
rz(1.3) q[2];

cx q[0], q[3];
rz(1.3) q[3];

cx q[0], q[4];
rz(1.3) q[4];

cx q[0], q[5];
rz(1.3) q[5];

cx q[0], q[6];
rz(1.3) q[6];

cx q[0], q[7];
rz(1.3) q[7];

cx q[0], q[8];
rz(1.3) q[8];

cx q[0], q[9];
rz(1.3) q[9];

cx q[0], q[10];
rz(1.3) q[10];

cx q[0], q[11];
rz(1.3) q[11];

cx q[0], q[12];
rz(1.3) q[12];

cx q[0], q[13];
rz(1.3) q[13];

cx q[0], q[14];
rz(1.3) q[14];

rz(1.3) q[0];

// Measurement
measure q[0] -> c[0];
measure q[1] -> c[1];
measure q[2] -> c[2];
measure q[3] -> c[3];
measure q[4] -> c[4];
measure q[5] -> c[5];
measure q[6] -> c[6];
measure q[7] -> c[7];
measure q[8] -> c[8];
measure q[9] -> c[9];
measure q[10] -> c[10];
measure q[11] -> c[11];
measure q[12] -> c[12];
measure q[13] -> c[13];
measure q[14] -> c[14];
'''

qasm_input_QAOA_25='''

OPENQASM 2.0;
include "qelib1.inc";

qreg q[25];
creg c[25];

// Initial state preparation: Apply Hadamard gates to all qubits
h q[0];
h q[1];
h q[2];
h q[3];
h q[4];
h q[5];
h q[6];
h q[7];
h q[8];
h q[9];
h q[10];
h q[11];
h q[12];
h q[13];
h q[14];
h q[15];
h q[16];
h q[17];
h q[18];
h q[19];
h q[20];
h q[21];
h q[22];
h q[23];
h q[24];

rz(0) q[0];

cx q[0], q[1];
rz(1) q[1];

cx q[0], q[2];
rz(2) q[2];

cx q[0], q[3];
rz(3) q[3];

cx q[0], q[4];
rz(1.3) q[4];

cx q[0], q[5];
rz(1.3) q[5];

cx q[0], q[6];
rz(1.3) q[6];

cx q[0], q[7];
rz(1.3) q[7];

cx q[0], q[8];
rz(1.3) q[8];

cx q[0], q[9];
rz(1.3) q[9];

cx q[0], q[10];
rz(1.3) q[10];

cx q[0], q[11];
rz(1.3) q[11];

cx q[0], q[12];
rz(1.3) q[12];

cx q[0], q[13];
rz(1.3) q[13];

cx q[0], q[14];
rz(1.3) q[14];

cx q[0], q[15];
rz(1.3) q[15];

cx q[0], q[16];
rz(1.3) q[16];

cx q[0], q[17];
rz(1.3) q[17];

cx q[0], q[18];
rz(1.3) q[18];

cx q[0], q[19];
rz(1.3) q[19];

cx q[0], q[20];
rz(1.3) q[20];

cx q[0], q[21];
rz(1.3) q[21];

cx q[0], q[22];
rz(1.3) q[22];

cx q[0], q[23];
rz(1.3) q[23];

cx q[0], q[24];
rz(1.3) q[24];


rz(1.3) q[0];

cx q[0], q[1];
rz(1.3) q[1];

cx q[0], q[2];
rz(1.3) q[2];

cx q[0], q[3];
rz(1.3) q[3];

cx q[0], q[4];
rz(1.3) q[4];

cx q[0], q[5];
rz(1.3) q[5];

cx q[0], q[6];
rz(1.3) q[6];

cx q[0], q[7];
rz(1.3) q[7];

cx q[0], q[8];
rz(1.3) q[8];

cx q[0], q[9];
rz(1.3) q[9];

cx q[0], q[10];
rz(1.3) q[10];

cx q[0], q[11];
rz(1.3) q[11];

cx q[0], q[12];
rz(1.3) q[12];

cx q[0], q[13];
rz(1.3) q[13];

cx q[0], q[14];
rz(1.3) q[14];

cx q[0], q[15];
rz(1.3) q[15];

cx q[0], q[16];
rz(1.3) q[16];

cx q[0], q[17];
rz(1.3) q[17];

cx q[0], q[18];
rz(1.3) q[18];

cx q[0], q[19];
rz(1.3) q[19];

cx q[0], q[20];
rz(1.3) q[20];

cx q[0], q[21];
rz(1.3) q[21];

cx q[0], q[22];
rz(1.3) q[22];

cx q[0], q[23];
rz(1.3) q[23];

cx q[0], q[24];
rz(1.3) q[24];

rz(1.3) q[0];

// Measurement
measure q[0] -> c[0];
measure q[1] -> c[1];
measure q[2] -> c[2];
measure q[3] -> c[3];
measure q[4] -> c[4];
measure q[5] -> c[5];
measure q[6] -> c[6];
measure q[7] -> c[7];
measure q[8] -> c[8];
measure q[9] -> c[9];
measure q[10] -> c[10];
measure q[11] -> c[11];
measure q[12] -> c[12];
measure q[13] -> c[13];
measure q[14] -> c[14];
measure q[15] -> c[15];
measure q[16] -> c[16];
measure q[17] -> c[17];
measure q[18] -> c[18];
measure q[19] -> c[19];
measure q[20] -> c[20];
measure q[21] -> c[21];
measure q[22] -> c[22];
measure q[23] -> c[23];
measure q[24] -> c[24];
'''

qasm_input_random_3='''
OPENQASM 2.0;
include "qelib1.inc";

qreg q[3];
creg c[3];

// Initial state preparation: Apply Hadamard gates to all qubits
x q[0];
x q[1];
x q[2];

h q[0];
cx q[0], q[2];
rz(1) q[1];

cx q[2], q[1];
rz(2) q[2];

rz(1.3) q[0];
cx q[0], q[1];

cx q[0], q[2];
x q[1];
x q[2];

x q[0];


// Measurement
measure q[0] -> c[0];
measure q[1] -> c[1];
measure q[2] -> c[2];
'''
qasm_input_random_5='''
OPENQASM 2.0;
include "qelib1.inc";

qreg q[5];
creg c[5];

// Initial state preparation: Apply Hadamard gates to all qubits
h q[0];
h q[1];
h q[2];
h q[3];
h q[4];

rz(0) q[0];
cx q[1], q[3];
rz(1) q[1];

cx q[2], q[3];
rz(2) q[2];

cx q[1], q[4];
rz(3) q[3];

cx q[2], q[4];
rz(1.3) q[4];

rz(1.3) q[0];
cx q[0], q[2];

cx q[2], q[3];

rz(1.3) q[1];
rz(1.3) q[2];

cx q[0], q[3];
rz(1.3) q[3];

cx q[0], q[4];
rz(1.3) q[4];
rz(1.3) q[0];

// Measurement
measure q[0] -> c[0];
measure q[1] -> c[1];
measure q[2] -> c[2];
measure q[3] -> c[3];
measure q[4] -> c[4];
'''
qasm_input_random_7='''
OPENQASM 2.0;
include "qelib1.inc";

qreg q[7];
creg c[7];

// Initial state preparation: Apply Hadamard gates to all qubits
h q[0];
h q[1];
h q[2];
h q[3];
h q[4];
h q[5];
h q[6];

rz(0) q[0];

cx q[0], q[2];
rz(1) q[1];

cx q[2], q[3];
rz(2) q[2];

cx q[1], q[5];
rz(3) q[3];

cx q[0], q[4];
rz(1.3) q[4];

cx q[3], q[5];
rz(1.3) q[5];

cx q[2], q[3];
rz(1.3) q[6];


rz(1.3) q[0];

cx q[0], q[2];
rz(1) q[1];

cx q[2], q[3];
rz(2) q[2];

cx q[1], q[5];
rz(3) q[3];

cx q[0], q[4];
rz(1.3) q[4];

cx q[3], q[5];
rz(1.3) q[5];

cx q[2], q[3];
rz(1.3) q[6];

rz(1.3) q[0];

// Measurement
measure q[0] -> c[0];
measure q[1] -> c[1];
measure q[2] -> c[2];
measure q[3] -> c[3];
measure q[4] -> c[4];
measure q[5] -> c[5];
measure q[6] -> c[6];
'''
qasm_input_random_9='''
OPENQASM 2.0;
include "qelib1.inc";

qreg q[9];
creg c[9];

// Initial state preparation: Apply Hadamard gates to all qubits
h q[0];
h q[1];
h q[2];
h q[3];
h q[4];
h q[5];
h q[6];
h q[7];
h q[8];

rz(0) q[0];

cx q[1], q[3];
rz(1) q[1];

cx q[4], q[8];
rz(2) q[2];

cx q[1], q[5];
rz(3) q[3];

cx q[2], q[6];
rz(1.3) q[4];

cx q[4], q[7];
rz(1.3) q[5];

cx q[2], q[3];
rz(1.3) q[6];

cx q[1], q[5];
rz(1.3) q[7];

cx q[6], q[8];
rz(1.3) q[8];


rz(1.3) q[0];

cx q[4], q[8];
rz(1.3) q[1];

cx q[3], q[7];
rz(1.3) q[2];

cx q[1], q[6];
rz(1.3) q[3];

cx q[0], q[4];
rz(1.3) q[4];

cx q[1], q[4];
rz(1.3) q[5];

cx q[5], q[7];
rz(1.3) q[6];

cx q[4], q[8];
rz(1.3) q[7];

cx q[3], q[5];
rz(1.3) q[8];


rz(1.3) q[0];

// Measurement
measure q[0] -> c[0];
measure q[1] -> c[1];
measure q[2] -> c[2];
measure q[3] -> c[3];
measure q[4] -> c[4];
measure q[5] -> c[5];
measure q[6] -> c[6];
measure q[7] -> c[7];
measure q[8] -> c[8];
'''
qasm_input_random_15='''

OPENQASM 2.0;
include "qelib1.inc";

qreg q[15];
creg c[15];

// Initial state preparation: Apply Hadamard gates to all qubits
h q[0];
h q[1];
h q[2];
h q[3];
h q[4];
h q[5];
h q[6];
h q[7];
h q[8];
h q[9];
h q[10];
h q[11];
h q[12];
h q[13];
h q[14];

rz(0) q[0];

cx q[5], q[10];
rz(1) q[1];

cx q[2], q[13];
rz(2) q[2];


cx q[2], q[4];
rz(3) q[3];

cx q[4], q[13];
rz(1.3) q[4];

cx q[0], q[5];
rz(1.3) q[5];

cx q[0], q[14];
rz(1.3) q[6];

cx q[2], q[7];
rz(1.3) q[7];

cx q[1], q[5];
rz(1.3) q[8];

cx q[4], q[12];
rz(1.3) q[9];

cx q[10], q[11];
rz(1.3) q[10];

cx q[0], q[4];
rz(1.3) q[11];

cx q[3], q[12];
rz(1.3) q[12];

cx q[7], q[9];
rz(1.3) q[13];

cx q[8], q[13];
rz(1.3) q[14];


rz(1.3) q[0];

cx q[5], q[10];
rz(1) q[1];

cx q[2], q[13];
rz(2) q[2];

cx q[2], q[13];
rz(3) q[3];

cx q[4], q[13];
rz(1.3) q[4];

cx q[0], q[5];
rz(1.3) q[5];

cx q[0], q[14];
rz(1.3) q[6];

cx q[2], q[7];
rz(1.3) q[7];

cx q[1], q[5];
rz(1.3) q[8];

cx q[4], q[12];
rz(1.3) q[9];

cx q[10], q[11];
rz(1.3) q[10];

cx q[0], q[4];
rz(1.3) q[11];

cx q[3], q[7];
rz(1.3) q[12];

cx q[7], q[9];
rz(1.3) q[13];

cx q[3], q[8];
rz(1.3) q[14];

rz(1.3) q[0];

// Measurement
measure q[0] -> c[0];
measure q[1] -> c[1];
measure q[2] -> c[2];
measure q[3] -> c[3];
measure q[4] -> c[4];
measure q[5] -> c[5];
measure q[6] -> c[6];
measure q[7] -> c[7];
measure q[8] -> c[8];
measure q[9] -> c[9];
measure q[10] -> c[10];
measure q[11] -> c[11];
measure q[12] -> c[12];
measure q[13] -> c[13];
measure q[14] -> c[14];
'''

qasm_input_random1_3='''

OPENQASM 2.0;
include "qelib1.inc";

qreg q[3];
creg c[3];

// Initial state preparation: Apply Hadamard gates to all qubits
x q[0];
x q[1];
x q[2];

h q[0];
cx q[0], q[2];
rz(1) q[1];

cx q[2], q[1];
rz(2) q[2];

rz(1.3) q[0];
cx q[0], q[2];

cx q[0], q[1];
x q[1];
x q[2];

x q[0];


// Measurement
measure q[0] -> c[0];
measure q[1] -> c[1];
measure q[2] -> c[2];
'''
qasm_input_random1_5='''
OPENQASM 2.0;
include "qelib1.inc";

qreg q[5];
creg c[5];

// Initial state preparation: Apply Hadamard gates to all qubits
h q[0];
h q[1];
h q[2];
h q[3];
h q[4];

rz(0) q[0];
cx q[1], q[2];
rz(1) q[1];

cx q[2], q[3];
rz(2) q[2];

cx q[1], q[4];
rz(3) q[3];

cx q[2], q[4];
rz(1.3) q[4];

rz(1.3) q[0];
cx q[0], q[2];

cx q[1], q[3];

rz(1.3) q[1];
rz(1.3) q[2];

cx q[0], q[2];
rz(1.3) q[3];

cx q[1], q[4];
rz(1.3) q[4];
rz(1.3) q[0];

// Measurement
measure q[0] -> c[0];
measure q[1] -> c[1];
measure q[2] -> c[2];
measure q[3] -> c[3];
measure q[4] -> c[4];
'''
qasm_input_random1_7='''
OPENQASM 2.0;
include "qelib1.inc";

qreg q[7];
creg c[7];

// Initial state preparation: Apply Hadamard gates to all qubits
h q[0];
h q[1];
h q[2];
h q[3];
h q[4];
h q[5];
h q[6];

rz(0) q[0];

cx q[0], q[2];
rz(1) q[1];

cx q[1], q[3];
rz(2) q[2];

cx q[1], q[4];
rz(3) q[3];

cx q[3], q[6];
rz(1.3) q[4];

cx q[3], q[5];
rz(1.3) q[5];

cx q[0], q[3];
rz(1.3) q[6];


rz(1.3) q[0];

cx q[0], q[4];
rz(1) q[1];

cx q[3], q[6];
rz(2) q[2];

cx q[2], q[6];
rz(3) q[3];

cx q[0], q[4];
rz(1.3) q[4];

cx q[2], q[5];
rz(1.3) q[5];

cx q[3], q[6];
rz(1.3) q[6];

rz(1.3) q[0];

// Measurement
measure q[0] -> c[0];
measure q[1] -> c[1];
measure q[2] -> c[2];
measure q[3] -> c[3];
measure q[4] -> c[4];
measure q[5] -> c[5];
measure q[6] -> c[6];
'''
qasm_input_random1_9='''
OPENQASM 2.0;
include "qelib1.inc";

qreg q[9];
creg c[9];

// Initial state preparation: Apply Hadamard gates to all qubits
h q[0];
h q[1];
h q[2];
h q[3];
h q[4];
h q[5];
h q[6];
h q[7];
h q[8];

rz(0) q[0];

cx q[1], q[4];
rz(1) q[1];

cx q[4], q[7];
rz(2) q[2];

cx q[3], q[5];
rz(3) q[3];

cx q[4], q[8];
rz(1.3) q[4];

cx q[5], q[6];
rz(1.3) q[5];

cx q[7], q[8];
rz(1.3) q[6];

cx q[5], q[6];
rz(1.3) q[7];

cx q[3], q[8];
rz(1.3) q[8];


rz(1.3) q[0];

cx q[4], q[5];
rz(1.3) q[1];

cx q[3], q[6];
rz(1.3) q[2];

cx q[1], q[4];
rz(1.3) q[3];

cx q[0], q[3];
rz(1.3) q[4];

cx q[1], q[2];
rz(1.3) q[5];

cx q[5], q[7];
rz(1.3) q[6];

cx q[3], q[4];
rz(1.3) q[7];

cx q[2], q[5];
rz(1.3) q[8];


rz(1.3) q[0];

// Measurement
measure q[0] -> c[0];
measure q[1] -> c[1];
measure q[2] -> c[2];
measure q[3] -> c[3];
measure q[4] -> c[4];
measure q[5] -> c[5];
measure q[6] -> c[6];
measure q[7] -> c[7];
measure q[8] -> c[8];
'''
qasm_input_random1_15='''

OPENQASM 2.0;
include "qelib1.inc";

qreg q[15];
creg c[15];

// Initial state preparation: Apply Hadamard gates to all qubits
h q[0];
h q[1];
h q[2];
h q[3];
h q[4];
h q[5];
h q[6];
h q[7];
h q[8];
h q[9];
h q[10];
h q[11];
h q[12];
h q[13];
h q[14];

rz(0) q[0];

cx q[3], q[10];
rz(1) q[1];

cx q[4], q[13];
rz(2) q[2];


cx q[6], q[14];
rz(3) q[3];

cx q[5], q[13];
rz(1.3) q[4];

cx q[0], q[5];
rz(1.3) q[5];

cx q[3], q[11];
rz(1.3) q[6];

cx q[2], q[7];
rz(1.3) q[7];

cx q[1], q[5];
rz(1.3) q[8];

cx q[10], q[12];
rz(1.3) q[9];

cx q[3], q[11];
rz(1.3) q[10];

cx q[0], q[4];
rz(1.3) q[11];

cx q[11], q[12];
rz(1.3) q[12];

cx q[7], q[9];
rz(1.3) q[13];

cx q[1], q[13];
rz(1.3) q[14];


rz(1.3) q[0];

cx q[3], q[10];
rz(1) q[1];

cx q[4], q[13];
rz(2) q[2];


cx q[6], q[14];
rz(3) q[3];

cx q[5], q[13];
rz(1.3) q[4];

cx q[0], q[5];
rz(1.3) q[5];

cx q[3], q[11];
rz(1.3) q[6];

cx q[2], q[7];
rz(1.3) q[7];

cx q[4], q[5];
rz(1.3) q[8];

cx q[10], q[12];
rz(1.3) q[9];

cx q[3], q[11];
rz(1.3) q[10];

cx q[0], q[4];
rz(1.3) q[11];

cx q[0], q[12];
rz(1.3) q[12];

cx q[7], q[9];
rz(1.3) q[13];

cx q[1], q[13];
rz(1.3) q[14];

rz(1.3) q[0];

// Measurement
measure q[0] -> c[0];
measure q[1] -> c[1];
measure q[2] -> c[2];
measure q[3] -> c[3];
measure q[4] -> c[4];
measure q[5] -> c[5];
measure q[6] -> c[6];
measure q[7] -> c[7];
measure q[8] -> c[8];
measure q[9] -> c[9];
measure q[10] -> c[10];
measure q[11] -> c[11];
measure q[12] -> c[12];
measure q[13] -> c[13];
measure q[14] -> c[14];
'''

qasm_input_random2_3='''

OPENQASM 2.0;
include "qelib1.inc";

qreg q[3];
creg c[3];

// Initial state preparation: Apply Hadamard gates to all qubits
x q[0];
x q[1];
x q[2];

h q[0];
cx q[0], q[2];
rz(1) q[1];

cx q[2], q[1];
rz(2) q[2];

rz(1.3) q[0];
cx q[0], q[2];

cx q[0], q[1];
x q[1];
x q[2];
cx q[2], q[1];
rz(2) q[2];

rz(1.3) q[0];
cx q[0], q[2];

cx q[0], q[1];
x q[0];


// Measurement
measure q[0] -> c[0];
measure q[1] -> c[1];
measure q[2] -> c[2];
'''
qasm_input_random2_5='''
OPENQASM 2.0;
include "qelib1.inc";

qreg q[5];
creg c[5];

// Initial state preparation: Apply Hadamard gates to all qubits
h q[0];
h q[1];
h q[2];
h q[3];
h q[4];

rz(0) q[0];
cx q[1], q[2];
rz(1) q[1];

cx q[2], q[4];
rz(2) q[2];

cx q[1], q[4];
rz(3) q[3];

cx q[2], q[3];
rz(1.3) q[4];

rz(1.3) q[0];
cx q[0], q[2];

cx q[1], q[3];

rz(1.3) q[1];
rz(1.3) q[2];

cx q[0], q[1];
rz(1.3) q[3];

cx q[1], q[3];
rz(1.3) q[4];
rz(1.3) q[0];

cx q[0], q[1];
rz(1.3) q[3];

cx q[0], q[3];

cx q[0], q[2];

// Measurement
measure q[0] -> c[0];
measure q[1] -> c[1];
measure q[2] -> c[2];
measure q[3] -> c[3];
measure q[4] -> c[4];
'''
qasm_input_random2_7='''
OPENQASM 2.0;
include "qelib1.inc";

qreg q[7];
creg c[7];

// Initial state preparation: Apply Hadamard gates to all qubits
h q[0];
h q[1];
h q[2];
h q[3];
h q[4];
h q[5];
h q[6];

rz(0) q[0];

cx q[0], q[2];
rz(1) q[1];

cx q[1], q[3];
rz(2) q[2];

cx q[1], q[4];
rz(3) q[3];

cx q[3], q[6];
rz(1.3) q[4];

cx q[3], q[5];
rz(1.3) q[5];

cx q[0], q[3];
rz(1.3) q[6];


rz(1.3) q[0];

cx q[2], q[4];
rz(1) q[1];

cx q[3], q[6];
rz(2) q[2];

cx q[4], q[6];
rz(3) q[3];

cx q[0], q[4];
rz(1.3) q[4];

cx q[1], q[5];
rz(1.3) q[5];

cx q[3], q[6];
rz(1.3) q[6];

rz(1.3) q[0];

cx q[2], q[6];
rz(3) q[3];

cx q[0], q[4];
rz(1.3) q[4];

cx q[4], q[5];
rz(1.3) q[5];

cx q[5], q[6];

// Measurement
measure q[0] -> c[0];
measure q[1] -> c[1];
measure q[2] -> c[2];
measure q[3] -> c[3];
measure q[4] -> c[4];
measure q[5] -> c[5];
measure q[6] -> c[6];
'''
qasm_input_random2_9='''
OPENQASM 2.0;
include "qelib1.inc";

qreg q[9];
creg c[9];

// Initial state preparation: Apply Hadamard gates to all qubits
h q[0];
h q[1];
h q[2];
h q[3];
h q[4];
h q[5];
h q[6];
h q[7];
h q[8];

rz(0) q[0];

cx q[1], q[4];
rz(1) q[1];

cx q[4], q[7];
rz(2) q[2];

cx q[3], q[5];
rz(3) q[3];

cx q[0], q[8];
rz(1.3) q[4];

cx q[2], q[6];
rz(1.3) q[5];

cx q[2], q[3];
rz(1.3) q[6];

cx q[2], q[6];
rz(1.3) q[7];

cx q[0], q[4];
rz(1.3) q[8];


rz(1.3) q[0];

cx q[4], q[5];
rz(1.3) q[1];

cx q[3], q[6];
rz(1.3) q[2];

cx q[0], q[4];
rz(1.3) q[3];

cx q[2], q[3];
rz(1.3) q[4];

cx q[1], q[2];
rz(1.3) q[5];

cx q[5], q[7];
rz(1.3) q[6];

cx q[0], q[4];
rz(1.3) q[7];

cx q[2], q[8];
rz(1.3) q[8];


rz(1.3) q[0];

cx q[0], q[1];
rz(1.3) q[3];

cx q[1], q[3];
rz(1.3) q[4];

cx q[1], q[6];
rz(1.3) q[5];

cx q[5], q[7];
rz(1.3) q[6];

cx q[0], q[4];
rz(1.3) q[7];

cx q[2], q[8];

// Measurement
measure q[0] -> c[0];
measure q[1] -> c[1];
measure q[2] -> c[2];
measure q[3] -> c[3];
measure q[4] -> c[4];
measure q[5] -> c[5];
measure q[6] -> c[6];
measure q[7] -> c[7];
measure q[8] -> c[8];
'''
qasm_input_random2_15='''

OPENQASM 2.0;
include "qelib1.inc";

qreg q[15];
creg c[15];

// Initial state preparation: Apply Hadamard gates to all qubits
h q[0];
h q[1];
h q[2];
h q[3];
h q[4];
h q[5];
h q[6];
h q[7];
h q[8];
h q[9];
h q[10];
h q[11];
h q[12];
h q[13];
h q[14];

rz(0) q[0];

cx q[3], q[10];
rz(1) q[1];

cx q[4], q[12];
rz(2) q[2];


cx q[8], q[14];
rz(3) q[3];

cx q[5], q[13];
rz(1.3) q[4];

cx q[0], q[5];
rz(1.3) q[5];

cx q[3], q[11];
rz(1.3) q[6];

cx q[2], q[7];
rz(1.3) q[7];

cx q[1], q[5];
rz(1.3) q[8];

cx q[10], q[12];
rz(1.3) q[9];

cx q[3], q[11];
rz(1.3) q[10];

cx q[0], q[4];
rz(1.3) q[11];

cx q[10], q[12];
rz(1.3) q[12];

cx q[7], q[9];
rz(1.3) q[13];

cx q[1], q[13];
rz(1.3) q[14];


rz(1.3) q[0];

cx q[4], q[10];
rz(1) q[1];

cx q[4], q[13];
rz(2) q[2];


cx q[7], q[14];
rz(3) q[3];

cx q[5], q[12];
rz(1.3) q[4];

cx q[0], q[5];
rz(1.3) q[5];

cx q[3], q[11];
rz(1.3) q[6];

cx q[2], q[7];
rz(1.3) q[7];

cx q[4], q[5];
rz(1.3) q[8];

cx q[10], q[12];
rz(1.3) q[9];

cx q[3], q[11];
rz(1.3) q[10];

cx q[0], q[4];
rz(1.3) q[11];

cx q[10], q[12];
rz(1.3) q[12];

cx q[7], q[9];
rz(1.3) q[13];

cx q[4], q[13];
rz(1.3) q[14];

rz(1.3) q[0];

cx q[2], q[9];
rz(1.3) q[13];

cx q[9], q[13];
rz(1.3) q[14];

cx q[7], q[14];
rz(1.3) q[13];

cx q[4], q[6];
rz(1.3) q[14];

// Measurement
measure q[0] -> c[0];
measure q[1] -> c[1];
measure q[2] -> c[2];
measure q[3] -> c[3];
measure q[4] -> c[4];
measure q[5] -> c[5];
measure q[6] -> c[6];
measure q[7] -> c[7];
measure q[8] -> c[8];
measure q[9] -> c[9];
measure q[10] -> c[10];
measure q[11] -> c[11];
measure q[12] -> c[12];
measure q[13] -> c[13];
measure q[14] -> c[14];
'''

In [5]:
solver = Heuristic("depth", "normal")


bowtie_connections = [(0,1),(1,2)]
bowtie_dev = qcdevice("bowtie", nqubits=3, connection=bowtie_connections, 
                  swap_duration=1)

solver.setdevice(bowtie_dev)

#parse a quasm file in this format with a parse function



circuit = QuantumCircuit()
circuit = circuit.from_qasm_str(str(qasm_input_random2_3))
#print(qasm_input1)
print(circuit)

simulator = AerSimulator()

# Run and get counts
result = simulator.run(circuit,shots=1).result()
counts = result.get_counts(circuit)
#plt.savefig('histogram.png')

#print_histo(counts)
solver.setprogram(qasm_input_random2_3)


result_depth, list_scheduled_gate_name, list_scheduled_gate_qubits,final_mapping,\
objective_value,qasm_output, compilation_time = solver.solve()

qasm_output_cnot=swap_to_cnot(qasm_output)

#print(qasm)
circuit1 = QuantumCircuit()
circuit1 = circuit1.from_qasm_str(str(qasm_output_cnot))
#print(circuit1)

start_time = datetime.datetime.now()
circuit2 = transpile(circuit1, optimization_level=3)
transpilation_time=datetime.datetime.now() - start_time
print(circuit2)


# Run and get counts
result1 = simulator.run(circuit2,shots=1).result()
counts1 = result1.get_counts(circuit2)
#print_histo(counts1)

print('depth before')
print(circuit.depth())

print('depth after')
print(circuit2.depth())

print(f'compilation_time = {compilation_time}')
print(f"transpilation time = {transpilation_time}.")

#KL divergence measure 2 distribuition
print('total variation distance: Mean_diff/2, max_diff, KL Divergence')
#print(compare_distribuitions(counts,counts1))

     ┌───┐  ┌───┐       ┌─────────┐                   ┌─────────┐              »
q_0: ┤ X ├──┤ H ├────■──┤ Rz(1.3) ├───────────■────■──┤ Rz(1.3) ├──────────────»
     ├───┤┌─┴───┴─┐  │  └──┬───┬──┘           │  ┌─┴─┐└──┬───┬──┘┌───┐         »
q_1: ┤ X ├┤ Rz(1) ├──┼─────┤ X ├──────────────┼──┤ X ├───┤ X ├───┤ X ├─────────»
     ├───┤└───────┘┌─┴─┐   └─┬─┘   ┌───────┐┌─┴─┐├───┤   └───┘   └─┬─┘┌───────┐»
q_2: ┤ X ├─────────┤ X ├─────■─────┤ Rz(2) ├┤ X ├┤ X ├─────────────■──┤ Rz(2) ├»
     └───┘         └───┘           └───────┘└───┘└───┘                └───────┘»
c: 3/══════════════════════════════════════════════════════════════════════════»
                                                                               »
«               ┌───┐┌─┐
«q_0: ──■────■──┤ X ├┤M├
«       │  ┌─┴─┐└┬─┬┘└╥┘
«q_1: ──┼──┤ X ├─┤M├──╫─
«     ┌─┴─┐└┬─┬┘ └╥┘  ║ 
«q_2: ┤ X ├─┤M├───╫───╫─
«     └───┘ └╥┘   ║   ║ 
«c: 3/═══════╩════╩═══╩═
«            2    1   0 
4
10
16
17
Compilation time = 0:00:00.000288.
